<a href="https://colab.research.google.com/github/Neeraj004/Projects/blob/main/Copy_of_Covid_19_Kerala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid 19 Prediction Kerala

In [1]:
#importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import plotly.express as px
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.layers import LSTM
from keras.optimizers import adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error
import math
import plotly.graph_objects as go

In [2]:
#loading Data
data=pd.read_csv("/content/drive/MyDrive/Covid 19 Confirmed Cases-Kerala.csv")
data.head()

,Date,Confirmed
0,2020-01-31,0.0
1,2020-02-01,0.0
2,2020-02-02,1.0
3,2020-02-03,1.0
4,2020-02-04,0.0


In [3]:
data['Date']= pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       841 non-null    datetime64[ns]
 1   Confirmed  841 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 13.3 KB


In [4]:
data.isna().sum()

Date         0
Confirmed    0
dtype: int64

In [5]:
#Ploting the data
px.line(data,x='Date',y="Confirmed")

In [33]:
np.random.seed(7)
df1=pd.DataFrame(data["Confirmed"])
df=df1.values
df[:5]

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.]])

In [34]:
#converting type to float and normalising values using minmax scaler
df=df.astype('float32')
scale=MinMaxScaler(feature_range=(0,1))
df=scale.fit_transform(df)

In [35]:
#spliting data into train and test with a ratio of 70:30
train,test=df[:int(len(df)*0.60),:],df[int(len(df)*0.60):,:]
print(len(train))
print(len(test))

504
337


In [36]:
#function to create dataset forcast data
def create_data(dataset,lookback):
  dataX,dataY=[],[]
  for i in range(len(dataset)-lookback-1):
    a=dataset[i:(i+lookback),0]
    dataX.append(a)
    dataY.append(dataset[i+lookback,0])
  return(np.array(dataX),np.array(dataY))

In [37]:
lookback=1
trainX,trainY=create_data(train,lookback)
testX,testY=create_data(test,lookback)

In [11]:
#building LSTM model
model=Sequential()
model.add(LSTM(4,input_shape=(1,lookback)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(trainX,trainY,epochs=10,batch_size=1,verbose=2)

Epoch 1/10
502/502 - 6s - loss: 0.0190 - 6s/epoch - 12ms/step
Epoch 2/10
502/502 - 1s - loss: 0.0123 - 1s/epoch - 3ms/step
Epoch 3/10
502/502 - 1s - loss: 0.0056 - 1s/epoch - 3ms/step
Epoch 4/10
502/502 - 1s - loss: 0.0020 - 1s/epoch - 3ms/step
Epoch 5/10
502/502 - 1s - loss: 0.0013 - 1s/epoch - 2ms/step
Epoch 6/10
502/502 - 2s - loss: 0.0013 - 2s/epoch - 5ms/step
Epoch 7/10
502/502 - 2s - loss: 0.0013 - 2s/epoch - 5ms/step
Epoch 8/10
502/502 - 2s - loss: 0.0013 - 2s/epoch - 3ms/step
Epoch 9/10
502/502 - 1s - loss: 0.0013 - 1s/epoch - 3ms/step
Epoch 10/10
502/502 - 2s - loss: 0.0014 - 2s/epoch - 3ms/step


In [12]:
#Testing the model
tr_pre=model.predict(trainX)
te_pre=model.predict(testX)
tr_pre=scale.inverse_transform(tr_pre)
trainY=scale.inverse_transform([trainY])
te_pre=scale.inverse_transform(te_pre)
testY=scale.inverse_transform([testY])

11/11 [==============================] - 0s 3ms/step


In [13]:
#finding mean squared error of actual value and predicted value
tr_score=math.sqrt(mean_squared_error(trainY[0],tr_pre[:,0]))
print("Train Score : %.2f RMSE"%(tr_score))
te_score=math.sqrt(mean_squared_error(testY[0],te_pre[:,0]))
print("Test Score : %.2f RMSE"%(te_score))
print('Train MAE : ',mean_absolute_error(trainY[0],tr_pre[:,0]))
print('Test MAE : ',mean_absolute_error(testY[0],te_pre[:,0]))
print('Train MAPE : ',mean_absolute_percentage_error(trainY[0],tr_pre[:,0]))
print('Test MAPE : ',mean_absolute_percentage_error(testY[0],te_pre[:,0]))

Train Score : 1969.76 RMSE
Test Score : 3373.12 RMSE
Train MAE :  1106.1824043602126
Test MAE :  1823.3745489902208
Train MAPE :  1.692307104284591e+17
Test MAPE :  0.35772061754517276


In [14]:
#creating a dataset for ploting
tr_plot=np.empty_like(df)
tr_plot[:,:]=np.nan
tr_plot[lookback:len(tr_pre)+lookback,:]=tr_pre
te_plot=np.empty_like(df)
te_plot[:,:]=np.nan
te_plot[len(tr_pre)+(lookback*2)+1:len(data)-1,:]=te_pre
x=data['Date']
y=list(scale.inverse_transform(df).ravel())
train=list(tr_plot.ravel())
test=list(te_plot.ravel())
df4=pd.DataFrame({'y':y,
                  'train':train,
                  'test':test},index=x)
df4.head()

,y,train,test
Date,,,
2020-01-31,0.0,NaN,NaN
2020-02-01,0.0,428.271301,NaN
2020-02-02,1.0,428.271301,NaN
2020-02-03,1.0,429.162018,NaN
2020-02-04,0.0,429.162018,NaN


In [15]:
#plot comparing actual data and predicted data
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['y'],
    name="Actual"
))

fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['train'],
    name="train_predicted"
))

fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['test'],
    name="test_predicted"
))
fig.update_layout( title="Covid 19 Forecast - LSTM")
fig.show()


# GRU

In [25]:
# Define the GRU model
model1 = Sequential()
model1.add(GRU(50, activation='tanh', input_shape=(1,lookback)))
model1.add(Dense(1))
model1.compile(optimizer='adam', loss='mse')

In [26]:
# Train the model
model1.fit(trainX,trainY,epochs=10,batch_size=1,verbose=2)

Epoch 1/10
502/502 - 3s - loss: 0.0074 - 3s/epoch - 6ms/step
Epoch 2/10
502/502 - 1s - loss: 0.0014 - 1s/epoch - 3ms/step
Epoch 3/10
502/502 - 1s - loss: 0.0014 - 1s/epoch - 3ms/step
Epoch 4/10
502/502 - 1s - loss: 0.0014 - 940ms/epoch - 2ms/step
Epoch 5/10
502/502 - 1s - loss: 0.0015 - 931ms/epoch - 2ms/step
Epoch 6/10
502/502 - 1s - loss: 0.0015 - 933ms/epoch - 2ms/step
Epoch 7/10
502/502 - 1s - loss: 0.0014 - 908ms/epoch - 2ms/step
Epoch 8/10
502/502 - 1s - loss: 0.0015 - 905ms/epoch - 2ms/step
Epoch 9/10
502/502 - 1s - loss: 0.0014 - 896ms/epoch - 2ms/step
Epoch 10/10
502/502 - 1s - loss: 0.0014 - 925ms/epoch - 2ms/step


In [38]:
# Evaluate the model
train_loss = model1.evaluate(trainX, trainY, verbose=2)
test_loss = model1.evaluate(testX, testY, verbose=2)
print('Train loss:', train_loss)
print('Test loss:', test_loss)

16/16 - 2s - loss: 0.0012 - 2s/epoch - 140ms/step
11/11 - 0s - loss: 0.0037 - 54ms/epoch - 5ms/step
Train loss: 0.0012480062432587147
Test loss: 0.003698617685586214


In [39]:
# Make predictions
tr_pre=model1.predict(trainX)
te_pre=model1.predict(testX)
tr_pre=scale.inverse_transform(tr_pre)
trainY=scale.inverse_transform([trainY])
te_pre=scale.inverse_transform(te_pre)
testY=scale.inverse_transform([testY])

11/11 [==============================] - 0s 2ms/step


In [40]:
#finding mean squared error of actual value and predicted value
tr_score=math.sqrt(mean_squared_error(trainY[0],tr_pre[:,0]))
print("Train Score : %.2f RMSE"%(tr_score))
te_score=math.sqrt(mean_squared_error(testY[0],te_pre[:,0]))
print("Test Score : %.2f RMSE"%(te_score))
print('Train MAE : ',mean_absolute_error(trainY[0],tr_pre[:,0]))
print('Test MAE : ',mean_absolute_error(testY[0],te_pre[:,0]))
print('Train MAPE : ',mean_absolute_percentage_error(trainY[0],tr_pre[:,0]))
print('Test MAPE : ',mean_absolute_percentage_error(testY[0],te_pre[:,0]))

Train Score : 1959.77 RMSE
Test Score : 3373.78 RMSE
Train MAE :  1032.1242651725706
Test MAE :  1791.1036140345618
Train MAPE :  9.03182575533422e+16
Test MAPE :  0.2628539719709132


In [41]:
#creating a dataset for ploting
tr_plot=np.empty_like(df)
tr_plot[:,:]=np.nan
tr_plot[lookback:len(tr_pre)+lookback,:]=tr_pre
te_plot=np.empty_like(df)
te_plot[:,:]=np.nan
te_plot[len(tr_pre)+(lookback*2)+1:len(data)-1,:]=te_pre
x=data['Date']
y=list(scale.inverse_transform(df).ravel())
train=list(tr_plot.ravel())
test=list(te_plot.ravel())
df4=pd.DataFrame({'y':y,
                  'train':train,
                  'test':test},index=x)
df4.head()

,y,train,test
Date,,,
2020-01-31,0.0,NaN,NaN
2020-02-01,0.0,228.323563,NaN
2020-02-02,1.0,228.323563,NaN
2020-02-03,1.0,229.287842,NaN
2020-02-04,0.0,229.287842,NaN


In [42]:
#plot comparing actual data and predicted data
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['y'],
    name="Actual",    
))

fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['train'],
    name="train_predicted"
))

fig.add_trace(go.Scatter(
    x=df4.index,
    y=df4['test'],
    name="test_predicted"
))
fig.update_layout( title="Covid 19 Forecast - GRU")
fig.show()
